In [2]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
result = search.run(
    "Research how to use the requests library in Python. Use your tools to search and summarize content into a guide on how to use the requests library."
)
print(result)

snippet: The Python requests library is a powerful tool for handling HTTP requests. Whether you're sending GET or POST requests or interacting with complex APIs, requests makes it all simple and intuitive. In this tutorial, you'll learn how to perform network requests, from basic to advanced usage, to help you master web programming., title: Python requests Library Tutorial - PerfCode, link: https://en.perfcode.com/python/requests, snippet: Python Requests Tutorial. The Requests library in Python is one of the integral parts of Python for making HTTP requests to a specified URL. Whether it be REST APIs or Web Scraping, requests are a must to be learned for proceeding further with these technologies. When one makes a request to a URI, it returns a response., title: Python Requests Tutorial - GeeksforGeeks, link: https://www.geeksforgeeks.org/python-requests-tutorial/, snippet: The Python requests module enables developers to write code to interact with REST APIs. It allows them to send 

In [5]:
# source https://dev.to/timesurgelabs/how-to-make-an-ai-agent-in-10-minutes-with-langchain-3i2n
import requests
from bs4 import BeautifulSoup
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import StructuredTool
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "{YOUR_LANGCHAIN_APIKEY}"
# os.environ["LANGCHAIN_PROJECT"] = "{YOUR_LANGSMITH_PROJECT}"
# os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"
# os.environ["SERPER_API_KEY"] = "{YOUR_SERPER_APIKEY}"

# model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ddg_search = DuckDuckGoSearchResults()
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0"
}


def parse_html(content) -> str:
    soup = BeautifulSoup(content, "html.parser")
    text_content_with_links = soup.get_text()[:3000]
    return text_content_with_links


def fetch_web_page(url: str) -> str:
    if isinstance(url, list):
        url = url[0]
    print("Fetch_web_page URL :", url)
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)


web_fetch_tool = StructuredTool.from_function(
    func=fetch_web_page,
    name="WebFetcher",
    description="Useful to fetches the contents of a web page",
)


summarization_chain = PromptTemplate.from_template("Summarize the following content: {content}") | model

summarize_tool = StructuredTool.from_function(
    func=summarization_chain.invoke,
    name="Summarizer",
    description="Useful to summarizes a web page",
)

tools = [ddg_search, web_fetch_tool, summarize_tool]

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=10,
)

question = "Tell me about best Korean reastaurant in Seoul.\
Use search tool to find the information.\
To get the details, please fetch the contents from the web sites.\
Summarize the details in 1000 words."

print(agent_executor.invoke({"input": question}))



> Entering new AgentExecutor chain...
I need to find information about the best Korean restaurants in Seoul. I'll start by using the search tool to gather relevant results. 

Action: duckduckgo_results_json  
Action Input: "best Korean restaurant in Seoul"  
snippet: Find the best places to eat in Seoul with this comprehensive guide to 25 restaurants serving authentic Korean dishes. From spicy jjukkumi to sannakji, from bibimbap to kalguksu, discover the diverse and delicious cuisine of Korea., title: 25 Seoul Restaurants You'll Want to Fly For | Will Fly for Food, link: https://www.willflyforfood.net/seoul-food-guide-25-must-eat-restaurants-in-seoul-south-korea/, snippet: Discover the best street food in Seoul, from tteokbokki to dakgangjeong, and where to find them in traditional markets, food trucks, and more. Learn useful Korean phrases to order street food and join fun food tours in Seoul., title: 30 Best Korean Street Food: Seoul Street Eats To Try In 2024 - In My Korea, link: 

In [6]:
print(web_fetch_tool.name)
print(web_fetch_tool.description)
print(web_fetch_tool.args)

WebFetcher
Useful to fetches the contents of a web page
{'url': {'title': 'Url', 'type': 'string'}}


In [7]:
import langchain
print(langchain.__version__)

0.3.4
